In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier

In [2]:
train = pd.read_csv("train.csv",delimiter = ",")
test = pd.read_csv("test.csv",delimiter = ",")

In [3]:
Y_train = train['airline_sentiment']
X_train = train.drop(['airline_sentiment','tweet_id','airline_sentiment_gold', 'negativereason_gold','tweet_coord'],axis = 1)
X_test = test.drop(['tweet_id','airline_sentiment_gold', 'negativereason_gold','tweet_coord'],axis = 1)

In [4]:
texts_train = train['text']
texts_test = test['text']

In [5]:
documents = [texts_train[i] for i in range(len(texts_train))]
documents_test = [texts_test[i] for i in range(len(texts_test))]

In [6]:
from nltk.tokenize import word_tokenize
words = []
for i in range(len(documents)):
    words.append((word_tokenize(documents[i]),Y_train[i]))


In [7]:
words_test = []
for i in range(len(documents_test)):
    words_test.append(word_tokenize(documents_test[i]))


In [8]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [9]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [10]:
from nltk import pos_tag

In [11]:
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
import string
punctuations = list(string.punctuation)
stops.update(punctuations)

In [12]:
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stops:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [13]:
documents = [(clean_review(word),category) for word,category in words]

In [14]:
documents_text = [(clean_review(word)) for word in words_test]


In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
categories = [category for document, category in documents]

In [17]:
test_text_documents = [" ".join(document) for document in documents_text]

In [18]:
text_documents = [" ".join(document) for document, category in documents]

In [19]:
count_vec = CountVectorizer(max_features = 2000,ngram_range=(1,2))
x_train_features = count_vec.fit_transform(text_documents)

In [20]:
x_test_features = count_vec.transform(test_text_documents)

In [21]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(x_train_features,Y_train)
preds_ = rf.predict(x_test_features)


In [22]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train_features,Y_train)
preds_lr = lr.predict(x_test_features)

In [23]:
np.savetxt("twitties1.csv",preds_lr,fmt = "%s")

In [24]:
from sklearn.metrics import accuracy_score


In [25]:

preds_sm = rf.predict(x_train_features)

In [26]:
rf.score(x_train_features,Y_train)

0.9765027322404372

In [27]:
emoticons=[":-)",":)", ":D",":o)",":]", ":3", ":c)",":>","=]","8)",">:[", ":-(", ":(", ":-c", ":c", ":-<", ":<", ":-[", 
":[", ":{" ,":-||", ":@", ">:(",":’-(", ":’(",">:O", ":-O", ":O", ":-o", ":o", "8O", "O_O", "o-o", "O_o", "o_O",
"o_o", "O-O" , ";)" ]